In [1]:
import xarray as xr
from fsspec import get_mapper
from gcsfs import GCSMap
from dask_kubernetes import KubeCluster
from dask.distributed import Client
from time import sleep

In [2]:
import warnings
warnings.filterwarnings("ignore")


In [3]:
cluster = KubeCluster(n_workers=5, silence_logs=50)
cluster.adapt(minimum=5, maximum=100, wait_count=10)

client = Client(cluster)
client

Client Scheduler: tcp://10.32.15.48:33711 Dashboard: /user/0000-0001-7783-5629/proxy/42463/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
bucket = f"gs://pangeo-data/eNATL60-BLBT02X-ssh"
ds=xr.open_zarr(GCSMap(bucket))


In [6]:
def get_gcsfs_version():
    import gcsfs
    return gcsfs.__version__

In [7]:
def get_fsspec_version():
    import fsspec
    return fsspec.__version__

In [8]:
get_gcsfs_version()


'0.3.1+4.g4d75ce1'

In [9]:
get_fsspec_version()

'0.5.2+15.gd774f0e'

In [10]:
client.submit(get_gcsfs_version).result()

'0.3.1+4.g4d75ce1'

In [11]:
client.submit(get_fsspec_version).result()

'0.5.2+15.gd774f0e'

In [ ]:
results = []
for n in range(20):
    
    vlist = [
        ds[vname].isnull().data.sum()
        for vname in ds.data_vars.keys()
    ]
    
    results.append(sum(vlist).compute(retries=10))
    
    print(n, results[-1])
    
    sleep(20) # allow for scaling down


0 240166551360
1 240166551360
2 240166551360
3 240166551360
4 240166551360
5 240249495360
6 240194199360
7 240166551360
8 240166551360
9 240166551360
10 240166551360
11 240166551360
12 240166551360
13 240166551360
14 240166551360
15 240166551360
